In [9]:
import re
import collections
from Bio import SeqIO
import os
from ete3 import Tree 
os.chdir('/Users/otoolea2/Documents/phd')
os.getcwd()

'/Users/otoolea2/Documents/phd'

In [2]:
translator_dict = {}
for my_sp in ['mac', 'gib', 'pongo', 'gorilla', 'chimp', 'homo']:
    with open('gid_tid_pid/'+my_sp+'_gid_tid_pid.txt','r') as f:
        for l in f:
            l = l.rstrip('\n')
            tokens = l.split(',')
            if tokens[2] != '':
                translator_dict[tokens[2]] = tokens[1]
print len(translator_dict)

226258


In [11]:
import subprocess

indir = 'duplicability/msa_files/new_dup/'
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.fa'):
            if my_file.startswith('pro'):
                subprocess.call(['muscle', '-in', r + '/'+ my_file, '-out', r + '/muscle_alignment.fa'])
                #cd += 1

indir = 'duplicability/msa_files/sing1/'
s1 = 0
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.fa'):
            if my_file.startswith('pro'):
                #print r + '/' + my_file
                subprocess.call(['muscle', '-in', r + '/'+ my_file, '-out', r + '/muscle_alignment.fa'])  
                #s1 += 1
                
print('finished sing1 '  + str(s1))  


indir = 'duplicability/msa_files/sing2/'
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.fa'):
            if my_file.startswith('pro'):
                subprocess.call(['muscle', '-in', r + '/'+ my_file, '-out', r + '/muscle_alignment.fa'])
                
print('finished sing2')                
                
indir = 'duplicability/msa_files/sing3/'
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.fa'):
            if my_file.startswith('pro'):
                subprocess.call(['muscle', '-in', r + '/'+ my_file, '-out', r + '/muscle_alignment.fa']) 
                
print('finished sing3')                


finished sing1 0
finished sing2
finished sing3


In [3]:
import os
import requests, sys
import re
from Bio import SeqIO
translator_dict = {}

for my_file in ['gib','mac', 'pongo','gorilla','chimp','homo']:
    with open('gid_tid_pid/'+ my_file + '_gid_tid_pid.txt','r') as f:
        for l in f:
            l = l.rstrip('\n')
            tokens = l.split(',')
            if tokens[2] != '':
                translator_dict[tokens[2]] = tokens[1]
print len(translator_dict)

server = "http://rest.ensembl.org"
ext = "/sequence/id/"

rootDir = './'
g = os.walk(rootDir)
dup_alignments = []
sing_alignments = [] 
c =0
for root, direct, files in g:
    if len(files) >= 2:
        my_requests = {}
        if os.path.exists(root+'/'+'tester_dup_file.txt'):

            for item in files:
                if item.endswith('protein_seqs.fa'):
                    
                    with open(root + '/' +item) as f_a:
                        for l in f_a:
                            if l.startswith('>'):
                                my_id = l.lstrip('>').rstrip('\n')
                                if my_id in translator_dict:
                                    my_id_a = translator_dict[my_id]
                                else:
                                    print 'error with ' + str(my_id)
                                #print my_id, my_id_a
                                r_a = requests.get(server+ext+my_id_a+"?type=cds", headers={ "Content-Type" : "text/x-fasta"})
                                #print r_a.text
                                if not r_a.ok:
                                    continue
                                    c+=1
                                    r_a.raise_for_status()
                                    sys.exit()
                                i = re.search("\n",r_a.text).start()    
                                
                            
                                if r_a.text[-4:-1] in ['TAA','TGA','TAG']:
                                    my_requests[my_id] = '>' + my_id + r_a.text[i:-4] + '\n'
                                else:
                                    my_requests[my_id] = '>' + my_id + r_a.text[i:] + '\n'

            
            f = open(root + '/' + 'nuc_seq.fa', 'w')
            
            if len(my_requests) > 2:
                for key in my_requests.keys():
                    f.write(str(my_requests[key]) + '\n')
            f.close()
print c            

226258
error with ENSP00000489519
0


In [4]:
server = "http://rest.ensembl.org"
ext = "/sequence/id/"

rootDir = './'
g = os.walk(rootDir)
dup_alignments = []
sing_alignments = [] 
c =0
ce = 0
for root, direct, files in g:
    if len(files) >= 2:
        my_requests = {}
        if os.path.exists(root+'/'+'tester_sing_file.txt'):

            for item in files:
                if item.endswith('protein_seqs.fa'):
                    
                    with open(root + '/' +item) as f_a:
                        for l in f_a:
                            if l.startswith('>'):
                                my_id = l.lstrip('>').rstrip('\n')
                                if my_id in translator_dict:
                                    my_id_a = translator_dict[my_id]
                                else:
                                    print 'error with ' + str(my_id)
                                    ce+=1
                                #print my_id, my_id_a
                                r_a = requests.get(server+ext+my_id_a+"?type=cds", headers={ "Content-Type" : "text/x-fasta"})
                                #print r_a.text
                                if not r_a.ok:
                                    continue
                                    c+=1
                                    r_a.raise_for_status()
                                    sys.exit()
                                i = re.search("\n",r_a.text).start()    
                                
                            
                                if r_a.text[-4:-1] in ['TAA','TGA','TAG']:
                                    my_requests[my_id] = '>' + my_id + r_a.text[i:-4] + '\n'
                                else:
                                    my_requests[my_id] = '>' + my_id + r_a.text[i:] + '\n'

            
            f = open(root + '/' + 'nuc_seq.fa', 'w')
            
            if len(my_requests) > 2:
                for key in my_requests.keys():
                    f.write(str(my_requests[key]) + '\n')
            f.close()
print c, ce   

0 0


In [9]:
from Bio import SeqIO

indir = 'duplicability/msa_files/new_dup/'
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.fa'):
            if my_file.startswith('pro'):
                handle = open(r+'/protein_seqs.fa','rU')
                proteins = {seq_record.id : len(seq_record.seq) for seq_record in SeqIO.parse(handle,"fasta")}
                handle.close()
                handle = open(r+'/nuc_seq.fa','rU')
                nuc = {seq_record.id : len(seq_record.seq) for seq_record in SeqIO.parse(handle,"fasta")}
                handle.close()
                for key in proteins:
                    if (3*proteins[key]) != (nuc[key]):
                        print r, key, proteins[key], nuc[key]
                        

In [10]:
from Bio import SeqIO

indir = 'duplicability/msa_files/sing/'
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.fa'):
            if my_file.startswith('pro'):
                handle = open(r+'/protein_seqs.fa','rU')
                proteins = {seq_record.id : len(seq_record.seq) for seq_record in SeqIO.parse(handle,"fasta")}
                handle.close()
                handle = open(r+'/nuc_seq.fa','rU')
                nuc = {seq_record.id : len(seq_record.seq) for seq_record in SeqIO.parse(handle,"fasta")}
                handle.close()
                for key in proteins:
                    if (3*proteins[key]) != (nuc[key]):
                        print r, key, proteins[key], nuc[key]

In [12]:
import os
import subprocess
 
rootDir = './'
g = os.walk(rootDir)
c=0
c2=0
for r, d, f in g:
    if os.path.exists(r+'/'+'muscle_alignment.fa'):
        #print(f)
        #print(os.path.exists(r+'/'+'needle.fa'))
        retcode = subprocess.call(['/Users/otoolea2/Documents/phd/Scripts/scripts_duplicability/translatorx_vLocal.pl','-i',r + '/nuc_seq.fa','-a',r + '/muscle_alignment.fa','-o',r + '/muscle_translatorx_res.nt_ali.fa'])
        if retcode != 0:
            print retcode
            print r
            c+=1
#     if os.path.exists(r + '/'+'tester_sing_file.txt'):
#         retcode = subprocess.call(['/Users/otoolea2/Documents/phd/Scripts/scripts_duplicability/translatorx_vLocal.pl','-i',r + '/nuc_seq.fa','-a',r + '/muscle_alignment.fa','-o',r + '/muscle_translatorx_res.nt_ali.fa'])
#         if retcode != 0:
#             print retcode
#             print r
#             c2+=1
print c

0
